# Creating derived variables using Tally, the hosted version of Quantipy

Tally is available on pip, to install it run

```
pip install datasmoothie-tally-client
```

If you are running this in gitpod, the python client has already been installed.

In [1]:
import tally
import os
import pandas as pd

## Working with different data sources

Tally works with SPSS, CSV files, the Confirmit API and Unicom/Dimensions files (mdd/ddf). Here we demonstrate using an SPSS file.

You need to get a Tally API key to run the example. Get in touch at info@datasmoothie.com if you need one. 

In [2]:
# we store the tally key in an environment variable, get in touch to get your own key
dataset = tally.DataSet(api_key=os.environ.get('tally_api_key'))
dataset.use_spss('data/Example Data (A).sav')

# also compatible with Confirmit, Nebu, Dimensions.

## Analysing meta data

We want to run a report on people who do sports/excersize regularly and have their main fitness activity as aerobics, yoga or pilates. In order to do this we look at the meta data for questions `q1` and `q2b` and find the codes for aerobics, yoga, pilates and regular sporties. These are **4, 5, 6** from `q1` and **1** for `q2b`.

In [3]:
# main sport activity
dataset.meta(variable='q1')

,codes,texts,missing
1,1,Swimming,None
2,2,Running/jogging,None
3,3,Lifting weights,None
4,4,Aerobics,None
5,5,Yoga,None
6,6,Pilates,None
7,7,Football (soccer),None
8,8,Basketball,None
9,9,Hockey,None
10,96,Other,None


In [4]:
# How regularly do you do sports
dataset.meta(variable='q2b')

,codes,texts,missing
1,1,Regularly,None
2,2,Irregularly,None
3,3,Never,None


## Create our logical conditions

We now have enough information to create our condition mapper. This maps out what code, label and logic is used to create each answer code for the new variable. 

In [5]:
cond_map = [
    (
        1, 
        "Regular sporties, mainly into yoga, aerobics or pilates", 
        { "$intersection": [{"q1":[4, 5, 6]}, {"q2b":[1]}] }
    ),
    (
        2, 
        "Non sporties, main activity not yoga, airobics, pilates", 
        {"$union":
            [
                {"$not_any":{"q2b":[1]}},
                {"$not_any":{"q1":[4,5,6]}}
            ]
        }
    )
]


We run derive with our new conditions to create the variable

In [6]:
dataset.derive(
    name='active_aer_yoga_pilates', 
    label='Active sporties into aerobics, yoga or pilates', 
    cond_maps=cond_map, 
    qtype="single"
)

And then we do a crosstab, to do a sanity check on the data.

In [7]:
dataset.crosstab(x='active_aer_yoga_pilates')

Question                                                                                            Total
Values                                                                                              Total
Question                                       Values                                                    
Active sporties into aerobics, yoga or pilates Base                                                8255.0
                                               Regular sporties, mainly into yoga, aerobics or...   339.0
                                               Non sporties, main activity not yoga, airobics,...  7916.0

In [8]:
main_vs_regular = dataset.crosstab(x='q1', y='q2b')

In [9]:
main_vs_regular.iloc[4:7,:]

Question                                     How regularly do you participate in any fitness or sports activity?  \
Values                                                                                                 Regularly   
Question                            Values                                                                         
What is your main fitness activity? Aerobics                                              277.0                    
                                    Yoga                                                   13.0                    
                                    Pilates                                                49.0                    

Question                                                         
Values                                       Irregularly  Never  
Question                            Values                       
What is your main fitness activity? Aerobics      1992.0  169.0  
                                    Yoga           152.0    6.0  
                                    Pilates        299.0   21.0

In [10]:
main_vs_regular.iloc[4:7,:].sum()

Question                                                             Values     
How regularly do you participate in any fitness or sports activity?  Regularly       339.0
                                                                     Irregularly    2443.0
                                                                     Never           196.0
dtype: float64

In [11]:
dataset.get_dataframe()[['q1', 'q2b', 'active_aer_yoga_pilates']].head()

,q1,q2b,active_aer_yoga_pilates
0,4.0,NaN,2.0
1,4.0,2.0,2.0
2,7.0,2.0,2.0
3,5.0,2.0,2.0
4,2.0,1.0,2.0


In [12]:
dataset.crosstab(x=['q14r01c01', 'q14r01c02'], y='active_aer_yoga_pilates', ci=['c%'])

Question                                                                            Active sporties into aerobics, yoga or pilates  \
Values                                                                     Regular sporties, mainly into yoga, aerobics or pilates   
Question                                        Values                                                                               
Store 1 - I had a good experience in the store. Base                                                                    339.0        
                                                Strongly disagree                                                        23.6        
                                                Disagree                                                                 23.6        
                                                Neither agree nor disagree                                               26.0        
                                                Agree                                                                    26.8        
                                                Strongly agree                                                            0.0        
Store 2 - I had a good experience in the store. Base                                                                    252.0        
                                                Strongly disagree                                                        27.8        
                                                Disagree                                                                 23.8        
                                                Neither agree nor disagree                                               22.6        
                                                Agree                                                                    25.8        
                                                Strongly agree                                                            0.0        

Question                                                                                                                            
Values                                                                     Non sporties, main activity not yoga, airobics, pilates  
Question                                        Values                                                                              
Store 1 - I had a good experience in the store. Base                                                                   7916.0       
                                                Strongly disagree                                                        24.6       
                                                Disagree                                                                 24.8       
                                                Neither agree nor disagree                                               25.8       
                                                Agree                                                                    24.8       
                                                Strongly agree                                                            0.0       
Store 2 - I had a good experience in the store. Base                                                                   5922.0       
                                                Strongly disagree                                                        25.1       
                                                Disagree                                                                 24.9       
                                                Neither agree nor disagree                                               24.9       
                                                Agree                                                                    25.1       
                                                Strongly agree                                                            0.0